<a href="https://colab.research.google.com/github/tomoya-ichikawa/NMT-LSTM-Attention-/blob/main/seq2seq_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

char2id = {str(i) : i for i in range(10)}


char2id.update({" ":10, "-":11, "_":12})

def generate_number():
  number = [random.choice(list("0123456789")) for _ in range(random.randint(1, 3))]
  return int("".join(number))

print(generate_number())

486


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm
import re
import collections
import itertools

In [ ]:
t = torch.tensor([[[1,2,3],
                   [4,5,6]],
                  [[0,0,0],
                   [1,1,1]]],dtype = torch.float64)

s = nn.Softmax(dim=2)
t = s(t)
t

tensor([[[0.0900, 0.2447, 0.6652],
         [0.0900, 0.2447, 0.6652]],

        [[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]]], dtype=torch.float64)

In [ ]:
t = torch.tensor([[[1,2,3],
                   [4,5,6]],
                  [[0,0,0],
                   [1,1,1]]],dtype = torch.float64)
t2 = torch.tensor([[[1,2,3],
                   [4,5,6]],
                  [[0,0,0],
                   [1,1,1]]],dtype = torch.float64)
t3 = torch.cat([t,t2], dim=2)
t3

tensor([[[1., 2., 3., 1., 2., 3.],
         [4., 5., 6., 4., 5., 6.]],

        [[0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1.]]], dtype=torch.float64)

In [ ]:


def normalizeString(s,l):
    if s == l[0]:
      s = s.lower().strip()
   
  
    s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def build_vocab(tokens):
  counts = collections.Counter(tokens)
  sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
  word_list = ["<PAD>","<SOS>", "<EOS>", "<UNK>"]+[x[0] for x in sorted_counts]
  word_dict = dict((w, i) for i, w in enumerate(word_list))
  return word_list, word_dict

def word2tensor(words, word_dict, max_len, trgg=False, padding=0):
  if trgg:
    words = ["<SOS>"] + words
  words = [word_dict.get(w, 3) for w in words]
  seq_len = len(words)
  if seq_len < max_len:
    words = words + [padding]*(max_len-seq_len)
  return torch.tensor(words, dtype=torch.int64), seq_len
def word2ltensor(words, word_dict, max_len, trgg=False, padding=0):
  if trgg:
    words =  words +["<EOS>"]
  words = [word_dict.get(w, 3) for w in words]
  seq_len = len(words)
  if seq_len < max_len:
    words = words + [padding]*(max_len-seq_len)
  return torch.tensor(words, dtype=torch.int64), seq_len


In [ ]:
class MyDataset(torch.utils.data.Dataset):

  def __init__(self, path, max_len=22, test=False):
    lines = open(path, encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s,l) for s in l.split('\t')] for l in lines]

    src = [p[0] for p in pairs]
    src = [l.split(" ") for l in src]
    trg = [p[1] for p in pairs]
    trg = [l.split(" ") for l in trg]
    if test:
      self.src_word_list, self.src_word_dict = train_data.src_word_list, train_data.src_word_dict
      self.trg_word_list, self.trg_word_dict = train_data.trg_word_list, train_data.trg_word_dict
    #リストを要素として持つリスト（2次元リスト）を平坦化する
    else:
      self.src_word_list,self.src_word_dict = build_vocab(itertools.chain.from_iterable(src))
      self.trg_word_list,self.trg_word_dict = build_vocab(itertools.chain.from_iterable(trg))

    self.src_data = [word2tensor(words, self.src_word_dict, max_len) for words in src]
    self.trg_data = [word2tensor(words, self.trg_word_dict, max_len, trgg=True) for words in trg]
    self.trg_label = [word2ltensor(words, self.trg_word_dict, max_len, trgg=True, padding = -100) for words in trg]

  def __len__(self):
    return len(self.trg_data)
  def __getitem__(self, idx):
    src, lsrc = self.src_data[idx]
    trg, ltrg = self.trg_data[idx]
    label, llabel = self.trg_label[idx]
    return src, lsrc, trg, ltrg,label, llabel

In [ ]:
org_list = ["a bc def gs"]
org = [l.split(" ") for l in org_list]
org

[['a', 'bc', 'def', 'gs']]

In [ ]:
batch_size =128
max_len =20
path = "/content/gdrive/My Drive/Colab-Notebooks/eng-ja.txt"
path2 = "/content/gdrive/My Drive/Colab-Notebooks/eng-ja2.txt"
train_data = MyDataset(path, max_len=max_len)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_data = MyDataset(path2, max_len=max_len, test=True)
test_loader = DataLoader(test_data)
test_data[0]

(tensor([  15,   33,    5, 1113,    7,   49,   14,  518,   23,   25,  861,    8,
            4,    0,    0,    0,    0,    0,    0,    0]),
 13,
 tensor([   1,   55,  781, 2449,   15,   40,  202,    4,    5,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 9,
 tensor([  55,  781, 2449,   15,   40,  202,    4,    5,    2, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]),
 9)

In [ ]:
train_data[0]
w = "".join(train_data.src_word_list[i] for i in test_data[0][0])
w2 = " ".join(train_data.trg_word_list[i] for i in train_data[0][4])
w

'彼らはついにそれが真実だと認めた。<PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [ ]:
train_data[0][0]

tensor([ 92,  14, 230,   7, 155,  29,  22,  18,   7,   5, 277,  38,  21,  41,
         30,   4,   0,   0,   0,   0])

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim=50,hidden_size=50, num_layer=1, dropout=0.1):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layer, batch_first=True, dropout=dropout)
  
  def forward(self, x, h0=None, l=None):
    x = self.emb(x)
    if l is not None:
      x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
    output, h = self.lstm(x,h0)
    if l is not None:
      output = nn.utils.rnn.pad_packed_sequence(output, batch_first=True, padding_value=0)[0]

    return output, h

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim=50, hidden_size=50, num_layer=1, dropout=0.1):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layer, batch_first=True, dropout=dropout)
    self.linear = nn.Linear(hidden_size*2, vocab_size)
    self.softmax = nn.Softmax(dim=2)

  def forward(self, x, h0=None, hs=None, l=None):
    x = self.emb(x)
    if l is not None:
      x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
    output, hidden = self.lstm(x,h0)
    if l is not None:
      output = nn.utils.rnn.pad_packed_sequence(output, batch_first=True, padding_value=0)[0]
    output_a = torch.transpose(output, 1, 2)
    s =torch.bmm(hs, output_a)
    s = torch.transpose(s, 1,2)
    attention_weight = self.softmax(s)
    attention_score = torch.bmm(attention_weight, hs)
    output = torch.cat([output, attention_score], dim=2)
    output = self.linear(output)
    return output, hidden

In [ ]:
enc = Encoder(len(train_data.src_word_list), 1024, 1024, 2)
dec = Decoder(len(train_data.trg_word_list), 1024, 1024, 2)
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)
loss_f = nn.CrossEntropyLoss()

In [ ]:
def to2D(x):
  shapes = x.shape
  return x.reshape(shapes[0]*shapes[1], -1)

In [ ]:
from statistics import mean

i =0
enc.train()
dec.train()
for epoc in range(20):
  losses = []
  for x, lx, y, ly, label, llabel in train_loader:
    lx, sort_idx = lx.sort(descending=True)
    x, y, ly, label, llabel= x[sort_idx], y[sort_idx], ly[sort_idx], label[sort_idx], llabel[sort_idx]
    x, y , label = x.to("cuda:0"), y.to("cuda:0"), label.to("cuda:0")
    
    out, ctx = enc(x, l=lx)
    ly, sort_idx = ly.sort(descending=True)
    y, label, llabel = y[sort_idx],label[sort_idx], llabel[sort_idx]
    h0 = (ctx[0][:,sort_idx,:], ctx[1][:, sort_idx, :])
    out = out[sort_idx,:,:]
    output,_ = dec(y, h0, out, l=ly)
    loss = loss_f(to2D(output), to2D(label[:, :max(ly)]).squeeze())
    enc.zero_grad()
    dec.zero_grad()
    loss.backward()
    opt_enc.step(), opt_dec.step()
    losses.append(loss.item())

  print(epoc, mean(losses))

  

0 2.6961849260208246
1 1.4129499206152718
2 0.9385520076507803
3 0.6670808899585549
4 0.5074161726343053
5 0.4004651921636918
6 0.3317499949651606
7 0.2920238571932249
8 0.26931584735050834
9 0.24517324825991754
10 0.22996226056000155
11 0.22015004576472066
12 0.21435044663946343
13 0.2135792345265903
14 0.2086882389643613
15 0.20698487850101402
16 0.19468268780680872
17 0.18268085913280088
18 0.17911451723417052
19 0.18038604284643822


In [ ]:
enc.eval()
dec.eval()
ptrgs=[]
with torch.no_grad():
  for x, lx, y, ly, label, llabel in test_loader:
    ptrg = translate(x, lx, enc, dec, device="cuda:0")
    y = y[:,1:ly]
    w = " ".join(train_data.trg_word_list[i] for i in y.squeeze())
    print(ptrg)
    ptrgs.append((ptrg, w))

ストリーミング出力は最後の 5000 行に切り捨てられました。
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])
torch.Size([])
torch.Size([1, 1, 6638])
torch.Size([1, 1, 6638])
torch.Size([6638])


KeyboardInterrupt: ignored

TypeError: ignored

In [ ]:
def translate(input_str, input_len, enc, dec, max_len=22, device = "cpu"):
  sos_inputs = torch.tensor(1, dtype=torch.int64)
  input_str = input_str.to(device)
  sos_inputs = sos_inputs.to(device)
  out, ctx = enc(input_str, l=input_len)
  z = sos_inputs
  results = []
  h= ctx
  for i in range(max_len):
    output, h = dec(z.view(1,1), h,out)
    print(output.size())
    wi = output.detach().view(-1).max(0)[1]
    print(output.detach().size())
    print(output.detach().view(-1).size())
    print(wi.size())
    if wi.item()==2:
      break
    results.append(wi.item())
    z = wi

  return " ".join(train_data.trg_word_list[i] for i in results)

In [ ]:
!ls

sample_data


In [ ]:
from google import colab
colab.drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

rnn = nn.LSTM(10, 20, 2,batch_first=True)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, hc = rnn(input)

In [ ]:
output.size()

torch.Size([5, 3, 20])

In [ ]:
hc[0].size()

torch.Size([2, 5, 20])

In [ ]:
import torch
a = torch.tensor([[[1, 3,53, 53,32],
      [1, 3,53, 53,32]],
     [[1, 3,53, 53,32],
      [1, 3,53, 53,32]]])
a[a==1] =0
a = to2D(a[:])
a



tensor([[ 0,  3, 53, 53, 32],
        [ 0,  3, 53, 53, 32],
        [ 0,  3, 53, 53, 32],
        [ 0,  3, 53, 53, 32]])